In [ ]:
sealed trait Nat
final object Zero extends Nat
final case class Suc(n: Nat) extends Nat
sealed trait List[+A]
final object Nil extends List[Nothing]
final case class Cons[A](head: A, tail: List[A]) extends List[A]

In [ ]:
def length[A]: List[A] => Int = {
    case Nil => 0
    case Cons(h, t) => 1 + length(t)
} 

In [ ]:
sealed trait Ring 
case object Zero extends Ring
case object One extends Ring
case class Elem(x: Int) extends Ring
case class Add(x: Ring, y: Ring) extends Ring
case class Mult(x: Ring, y: Ring) extends Ring


In [ ]:
def reduce: Ring => Ring = {
    case Zero => Zero
    case One => One
    case Elem(x) => Elem(x)
    case Add(x, y) => if (x == y) Mult(reduce(x), Elem(2)) else Add(reduce(x), reduce(y))
    case Mult(x, y) => Mult(reduce(x), reduce(y))
}

In [ ]:
def toInt: Ring => Int = {
    case Zero => 0
    case One => 1
    case Elem(x) => x
    case Add(x, y) => toInt(x) + toInt(y)
    case Mult(x, y) => toInt(x) * toInt(y)
}

In [ ]:
sealed trait RingF[+A]
case object Zero extends RingF[Nothing]
case object One extends RingF[Nothing]
case class Elem(x: Int) extends RingF[Nothing]
case class Add[A](x: A, y: A) extends RingF[A]
case class Mult[A](x:A, y: A) extends RingF[A]



In [ ]:
val expresion1 = Mult(Elem(4), Add(Elem(3), One))

In [ ]:
def evalToInt: RingF[Int] => Int = {
    x => x match {
        case Zero => 0
        case One => 1
        case Elem(x) => x
        case Add(x, y) => x + y
        case Mult(x, y) => x * y
    }
}



In [ ]:

trait Functor[F[_]] {
    def map[A,B](f: A => B): F[A] => F[B]
}

implicit val ringFunctor = new Functor[RingF] {
    override def map[A, B](f: A => B): RingF[A] => RingF[B] = {
        case Zero => Zero
        case One => One
        case Elem(x) => Elem(x)
        case Add(x, y) => Add(f(x), f(y))
        case Mult(x, y) => Mult(f(x), f(y))
    }
}

In [ ]:
def liftInt: RingF[RingF[Int]] => RingF[Int] = {
    ringFunctor.map(evalToInt)
}

In [ ]:
val rare = Add(Elem(1), Elem(2))
liftInt(rare)

In [ ]:
case class Fix[F[_]](value: F[Fix[F]] )
object Fix {
  def fix[F[_]](ff: F[Fix[F]]): Fix[F] = new Fix[F](ff)
  def unfix[F[_]]: Fix[F] => F[Fix[F]] = f => f.value
}

In [ ]:
type Ring = Fix[RingF]
val zero = Fix[RingF](Zero)
val one =  Fix[RingF](One)
def elem: Int  => Ring = x => Fix[RingF](Elem(x))
def add: (Ring, Ring) => Ring = (x, y) =>  Fix[RingF](Add(x, y))
def mult: (Ring, Ring) => Ring = (x, y) =>  Fix[RingF](Mult(x, y))

In [ ]:
val s = add(elem(7), elem(7))
val exp1 = mult(elem(4), add(elem(3), one))

In [ ]:
def cata: Fix[RingF] => Int = {
    x => (evalToInt compose ringFunctor.map(cata) compose Fix.unfix)(x)
}


In [ ]:
cata(exp1)

In [ ]:
def cata[F[_], A](alg: F[A] => A)(implicit F: Functor[F]): Fix[F] => A = {
    x => alg(F.map(cata(alg))(Fix.unfix(x)))
}

In [ ]:
def cata2[F[_], A](alg: F[A] => A)(implicit F: Functor[F]): Fix[F] => A = {
    x => (alg compose F.map(cata(alg)) compose Fix.unfix)(x)
}

In [ ]:
cata2(evalToInt)(ringFunctor)(exp1)

In [ ]:
def recursiveEvalToInt = cata(evalToInt)
recursiveEvalToInt(exp1)

In [ ]:
/**h
Repasar esto para que funcione
**/
sealed trait ListF[+A, +B]
sealed trait List[+A] extends ListF[A, Fix[List]]
final object Nil extends ListF[Nothing, Nothing]
final case class Cons[A, B](head: A, tail: B) extends ListF[A, B]

In [ ]:
def findDivisorsOf(r: Int): RingF[Int] = {
    def loop(n: Int): RingF[Int] = {
        if (n >= r/2) Elem(r)
        else if (r%n == 0) Mult(r/n, n) 
        else loop(n+1)
    }
    r match {
        case 0 => Zero
        case 1 => One
        case _ => loop(2)
    }
}

In [ ]:
findDivisorsOf(6)
findDivisorsOf(12)
findDivisorsOf(1)
findDivisorsOf(3)

In [ ]:
def f1: RingF[Int] => RingF[RingF[Int]] = ringFunctor.map(findDivisorsOf)
def find: Int => RingF[RingF[Int]] = {
    x => f1(findDivisorsOf(x))
}

In [ ]:
find(8)

In [ ]:
def ana[F[_], A](coalg: A => F[A])(implicit F: Functor[F]): A => Fix[F] = {
    x => Fix.fix((F.map(ana(coalg)) compose coalg)(x))
}

In [ ]:
val expression3 = ana(findDivisorsOf)(ringFunctor)(12)

In [ ]:
def evalToStr: RingF[String] => String = {
    x => x match {
        case Zero => "0"
        case One => "1"
        case Elem(x) => x.toString
        case Add(x, y) => x.toString + " + " + y.toString
        case Mult(x, y) => x.toString + " * " + y.toString
    }
}

In [ ]:
cata(evalToStr)(ringFunctor)(ana(findDivisorsOf)(ringFunctor)(12))

In [ ]:
import $ivy.`io.github.stanch::reftree:1.3.0`

{{
  import reftree.render._, reftree.diagram._
  import reftree.contrib.SimplifiedInstances.string
  val renderer = Renderer(renderingOptions = RenderingOptions(density = 60))
    
  renderer.render("examples/example5", Diagram.sourceCodeCaption(expression3))
}}
Image.fromFile("examples/example5.png")

In [ ]:
sealed trait ListF[+A] 
case object NilF extends ListF[Nothing]
case class ConsF[A](head: String, tail: A) extends ListF[A]

In [ ]:
val listOfStringFunctor = new Functor[ListF] {
 override def map[A, B](f: A => B): ListF[A] => ListF[B] = {
     case NilF => NilF
     case ConsF(h, t) => ConsF(h, f(t))
 }   
}

In [ ]:
type ListString = Fix[ListF]
val nil = Fix[ListF](NilF)
def cons : (String, ListString) => ListString = (h, t) =>  Fix[ListF](ConsF(h, t))

In [ ]:
val fixL1 = cons("a", cons("b", cons("c", nil)))

In [ ]:
def length: ListF[Int] => Int = {
    case NilF => 0
    case ConsF(h, t) => t + 1
}

cata(length)(listOfStringFunctor)(fixL1)

In [ ]:
val l1 = List("a", "b", "c")

In [ ]:
import $ivy.`io.github.stanch::reftree:1.3.0`

{{
  import reftree.render._, reftree.diagram._
  import reftree.contrib.SimplifiedInstances.string
  val renderer = Renderer(renderingOptions = RenderingOptions(density = 60))
    
  renderer.render("examples/example7", Diagram.sourceCodeCaption(l1))
}}
Image.fromFile("examples/example7.png")

In [ ]:
l1.foldLeft(0)((h, t) => h + 1)

In [ ]:
import $ivy.`io.github.stanch::reftree:1.3.0`

{{
  import reftree.render._, reftree.diagram._
  import reftree.contrib.SimplifiedInstances.string
  val renderer = Renderer(renderingOptions = RenderingOptions(density = 60))
    
  renderer.render("examples/example8", Diagram.sourceCodeCaption(fixL1))
}}
Image.fromFile("examples/example8.png")

In [ ]:
def myFoldLeft[A](baseValue: A)(evaluator: (String, A) => A)(l: ListString): A = {
    def alg: ListF[A] => A = {
        case NilF => baseValue
        case ConsF(h, t) => evaluator(h, t)
        }
    cata(alg)(listOfStringFunctor)(l)

}

In [ ]:
myFoldLeft(0)((x, y) => y + 1)(fixL1)

In [ ]:
def hylo[A, B, F[_]](ev1: A => F[A])(ev2: F[B] => B)(implicit functor: Functor[F]): A => B = {
    x => cata(ev2)(functor)((ana(ev1)(functor)(x)))
}

In [ ]:
hylo(findDivisorsOf)(evalToInt)(ringFunctor)(12)

In [ ]:
def prettyPrint: RingF[String] => String = {
    case Zero => "0"
    case One => "1"
    case Elem(x: Int) => x.toString
    case Add(x, y) => x + " + " + y
    case Mult(x, y) => x + " * " + y
}

In [ ]:
hylo(findDivisorsOf)(prettyPrint)(ringFunctor)(12)

In [ ]:
def optimize[A]: RingF[Fix[RingF]] => Fix[RingF] = {
    case Add(x, y) => if (x == y) Fix[RingF](Mult(y, Fix[RingF](Elem(2)))) else Fix[RingF](Add(x,y))
    case other => Fix[RingF](other)
}

In [ ]:
val a = add(add(elem(3), elem(3)), add(elem(3), elem(3)))

In [ ]:
cata(prettyPrint)(ringFunctor)(cata(optimize)(ringFunctor)(a))

In [ ]:
sealed trait LList[+A, +B]
object  NilL extends LList[Nothing, Nothing]
case class ConsL[A, B](h: A, t: B) extends LList[A, B]

In [ ]:
case class Fix2[F[_, _]](value: F[_, Fix2[F]])

In [ ]:
type Mlist = Fix2[LList]
val nil = Fix2[LList](NilL)
def cons[A]: (A, Mlist) => Mlist = (x, y) => Fix2[LList](ConsL(x, y))

In [ ]:
type Pol = List[Int]


sealed trait RingF[+A, +B]
case object Zero extends RingF[Nothing, Nothing]
case object One extends RingF[Nothing, Nothing]
//case class Elem[B](x: B) extends RingF[Nothing, B]
//case class Add[A, B](x: A, y: A) extends RingF[A, B]
//case class Mult[A, B](x:A, y: A) extends RingF[A, B]
//
def evPol: RingF[List[Int], List[Int]] => List[Int] = {
    case Zero => List[0]
    case One => List(1)
}


In [58]:
List(0)==List[Int](0)

res57: Boolean = true

In [52]:
cons("A", cons(7, nil))

res51: Mlist = Fix2(
  ConsL("A", Fix2(ConsL(7, Fix2(ammonite.$sess.cmd48$Helper$NilL$@53c9a54e))))
)

In [53]:
import reftree.render._
import reftree.diagram._
import java.nio.file.Paths

val renderer = Renderer(
  renderingOptions = RenderingOptions(density = 60),
  directory = Paths.get("examples", "guide")
)
import scala.collection.immutable.Queue


import reftree.render._

import reftree.diagram._

import java.nio.file.Paths


renderer: Renderer = Renderer(
  RenderingOptions(
    0.8,
    WrappedArray(
      RGBA(0.06274509803921569, 0.3058823529411765, 0.5450980392156862, 1.0),
      RGBA(0.13333333333333333, 0.5450980392156862, 0.13333333333333333, 1.0),
      RGBA(0.803921568627451, 0.3568627450980392, 0.27058823529411763, 1.0)
    ),
    "Source Code Pro",
    60
  ),
  AnimationOptions(2 seconds, 1 second, 9, true, 0),
  examples/guide,
  "png"
)
import scala.collection.immutable.Queue


In [52]:
renderer.render("diagram2",Diagram(Queue(1)).withCaption("one") + Diagram(Queue(2)).withCaption("two"))

In [53]:
val z = mult(elem(2), add(elem(2), elem(3)))
//renderer.render("animation2", Animation.startWith(z.value).iterate(x => x).build())

z: Ring = Fix(Mult(Fix(Elem(2)), Fix(Add(Fix(Elem(2)), Fix(Elem(3))))))

In [54]:
def optimiz: RingF[Fix[RingF]] => Fix[RingF] = {
    case Add(x, y) => x match {
        case Fix(Elem(z)) => y match {
            case Fix(Elem(w)) =>Fix[RingF](Elem(z+w))
            case Fix(other) => Fix[RingF](Add(x, optimiz(y.value)))
        }
        case Fix(other) => optimiz(other)
    }
    case Mult(x, y) => x match {
        case Fix(Elem(z)) => y match {
            case Fix(Elem(w)) =>Fix[RingF](Elem(z*w))
            case Fix(other) => Fix[RingF](Mult(x, optimiz(y.value)))
        }
        case Fix(other) => optimiz(other)
    }
    case other => optimiz(other)
}

def unoptimize: RingF[Fix[RingF]] => Fix[RingF] = {
    case Elem(x) => {
        val y = findDivisorsOf(x)
        y match{ 
            case Elem(z) => elem(z)
            case Mult(a, b) => mult(elem(a), elem(b))
        }
        
    }
    case Mult(x, y) => x match {
        case Fix(Elem(z)) => mult(unoptimize(Elem(z)), y)
        //case Fix(other) => unoptimize(other)
    }
    y match {
        case Fix(Elem(z)) => unoptimize(Elem(z))
        case Fix(other) => unoptimize(other)
    }
    case other => unoptimize(other)
}

defined function optimiz
defined function unoptimize

In [55]:
def iterateoptimize(n: Int): Fix[RingF] => Fix[RingF] = {
    if (n == 0) x => optimiz(x.value)
    else x => iterateoptimize(n-1)(optimiz(x.value))
}


defined function iterateoptimize

In [56]:
val exp1 = mult(elem(4), add(elem(3), elem(1)))

exp1: Ring = Fix(Mult(Fix(Elem(4)), Fix(Add(Fix(Elem(3)), Fix(Elem(1))))))

In [57]:
renderer.render("animation", Animation.startWith(exp1).iterate(2)(x => iterateoptimize(0)(x)).build())

In [58]:
iterateoptimize(0)(z)

res57: Fix[RingF] = Fix(Mult(Fix(Elem(2)), Fix(Elem(5))))

In [59]:
unoptimize(Elem(16))

res58: Fix[RingF] = Fix(Mult(Fix(Elem(8)), Fix(Elem(2))))

In [59]:
//def foldr[A, B](l: List[A])(seed: B)(evaluator: (A, B) => B): B  = {
//    l match {
//        case Nil => seed
//        case Cons(h, t) => evaluator(h, foldr(t)(seed)(evaluator))
//    }
//}

In [60]:
def foldR[Z](e: Fix[RingF])(elem: Elem => Z)(add: (Z, Z) => Z)(mult: (Z, Z) => Z): Z = {
    e.value match {
        case Elem(x)  => elem(Elem(x))
        case Add(x, y) => add(foldR(x)(elem)(add)(mult), foldR(y)(elem)(add)(mult))
        case Mult(x, y) => mult(foldR(x)(elem)(add)(mult), foldR(y)(elem)(add)(mult))
    }
}

defined function foldR

In [61]:
a

res60: Ring = Fix(
  Add(
    Fix(Add(Fix(Elem(3)), Fix(Elem(3)))),
    Fix(Add(Fix(Elem(3)), Fix(Elem(3))))
  )
)

In [62]:
foldR[Int](a)(x => x.x)((x, y) => x + y)((x,y) => x * y)

res61: Int = 12

In [63]:
def myFoldR[Z](a: Fix[RingF])(alg: RingF [Z] => Z):Z = {
    cata(alg)(ringFunctor)(a)
}

defined function myFoldR

In [64]:
myFoldR(a)(evalToInt)

res63: Int = 12

In [65]:
trait Recursive[F[_]] extends Functor[F] {
    def cata[A](alg: F[A] => A)(x: Fix[F])(implicit F: Functor[F]): A = {
    alg(F.map(cata(alg))(Fix.unfix(x)))
    }
}

val Ring = new Recursive[RingF] {
    implicit val ff = ringFunctor
    override def map[A, B](f: A => B): RingF[A] => RingF[B] = {
        case Zero => Zero
        case One => One
        case Elem(x) => Elem(x)
        case Add(x, y) => Add(f(x), f(y))
        case Mult(x, y) => Mult(f(x), f(y))
    }
}

defined trait Recursive
Ring: AnyRef with Recursive[RingF]{implicit val ff: cmd64.this.cmd8.Functor[ammonite.$sess.cmd8.wrapper.cmd5.RingF]} = ammonite.$sess.cmd64$Helper$$anon$1@4a6d2097

In [66]:
Ring.cata(evalToInt)(a)

res65: Int = 12

In [67]:
type Alg[Z] = RingF[Z] => Z
def myAlg: Alg[Fix[RingF]] = {
            case Zero => Fix[RingF](Zero)
            case One => Fix[RingF](One)
            case Elem(x) => Fix[RingF](Elem(x))
            case Add(x, y) => Fix[RingF](Add(x, y))
            case Mult(x, y) => Fix[RingF](Mult(x, y))
        }

defined type Alg
defined function myAlg

In [68]:
trait BBEnc {
    def apply[Z](alg: Alg[Z]): Z
}

defined trait BBEnc

In [69]:
trait Iso[A, B] {
    def to(a: A): B
    def from(b: B): A
}

defined trait Iso

In [70]:
val RingIso = new Iso[Fix[RingF], BBEnc] {
    override def to(expr: Fix[RingF]): BBEnc = {
        new BBEnc {
            override def apply[Z](alg: Alg[Z]): Z = {
                cata(alg)(ringFunctor)(expr)
            }
        }
    }
    
    override def from(enc: BBEnc): Fix[RingF] = {
        def myAlg: Alg[Fix[RingF]] = {
            case Zero => Fix[RingF](Zero)
            case One => Fix[RingF](One)
            case Elem(x) => Fix[RingF](Elem(x))
            case Add(x, y) => Fix[RingF](Add(x, y))
            case Mult(x, y) => Fix[RingF](Mult(x, y))
        }
        
        enc.apply(myAlg)

    }
}

RingIso: AnyRef with Iso[Fix[RingF], BBEnc] = ammonite.$sess.cmd69$Helper$$anon$1@61c5187e

In [73]:
exp1

res72: Ring = Fix(Mult(Fix(Elem(4)), Fix(Add(Fix(Elem(3)), Fix(Elem(1))))))

In [74]:
RingIso.from(RingIso.to(exp1)) == exp1

res73: Boolean = true